In [ ]:
# dé-commenter cette ligne si nbtutor n'est pas installé
%load_ext nbtutor

In [ ]:
from IPython.core.display import HTML

def _set_css_style(css_file_path):
   """
   Read the custom CSS file and load it into Jupyter.
   Pass the file path to the CSS file.
   """

   styles = open(css_file_path, "r").read()
   s = '<style>%s</style>' % styles     
   return HTML(s)

_set_css_style('custom/custom.css')

<div style='width:20em'><img src='img/logo-igm.png'></div>
<div style='font-size:larger'><strong>Algorithmique et programmation 2</strong><br>
L1 Mathématiques - L1 Informatique<br>
Semestre 2
</div>

# Récursivité

## Appels de fonctions et pile d'appels

### Une petite histoire

- Vous allez chez G. en lisant un bouquin  
    - Arrivée à la station, vous commencez à consulter le plan  
        - R. appelle pour vous proposer un cinéma demain  
            - Vous consultez le programme  
            - Vous trouvez un film qui vous plaît  
        - Vous vous mettez d'accord avec R.  
    - Vous finissez de chercher votre ligne de métro  
- Vous reprenez votre lecture à la bonne page

Un être humain sait faire ça (un peu), **mais une machine** ?

### Les appels de fonctions
  
  -   Un programme utilise des fonctions (pré-définies ou non)
    <!-- $1 -->
    <!-- $1 -->
    <!-- $1 -->
    <!-- $1 -->
    <!-- $1 -->
    <!-- $1 -->
    <!-- $1 -->
    <!-- $1 -->
  -   Certaines fonctions font appel à d'autres fonctions
    <!-- $1 -->
    <!-- $1 -->
    <!-- $1 -->
    <!-- $1 -->
    <!-- $1 -->
  -   Au moment de l'appel, l'exécution de la fonction appelante
    s'interrompt
  -   Après un appel, l'exécution doit reprendre au bon endroit, et
    avec les bonnes valeurs des variables
  

<img src='img/appelrec.png' width=50%></img>

### Exemple : quelques tâches simples

**Exercice :**
Décrivez l'exécution du programme suivant, l'évolution de la pile
d'appels et les affichages

In [ ]:
%%nbtutor -r -f -d 10

def pgcd(a, b):
    while b:
        a, b = b, a % b
    return a

def simplifier(fraction):
    d = pgcd(fraction[0], fraction[1])
    res = (fraction[0] // d, fraction[1] // d)
    return res

def ajouter(frac1, frac2):
    denom = frac1[1] * frac2[1]
    num = frac1[0] * frac2[1] + frac2[0] * frac1[1]
    res = simplifier((num, denom))
    return res

frac = ajouter((1, 3), (1, 6))

### Pile d'appels

**Pile d'appels :** liste ordonnée de tâches en cours
  
-   « Fond » de la pile : première tâche commencée (la plus ancienne)
-   « Sommet » de la pile : dernière tâche commencée (la plus récente)  

  

  Chaque élément de pile <!-- $1 -->
  contient :
  
  -   Valeur des paramètres de la fonction en cours
  -   Position du site d'appel (instruction à exécuter au retour)
  -   Variables locales à la fonction, état de l'interpréteur
  

### Fonctionnement de la pile

  À chaque nouvel **appel** :
  -   La fonction appelante évalue chaque paramètre et met sur la pile (des références à) ces valeurs
  -   L'appelante sauvegarde son état (ou *contexte*) pour
    pouvoir reprendre son exécution une fois l'appel terminé
  -   L'interpréteur donne la main à la fonction appelée
  -   L'appelée démarre son exécution, en plaçant ses éventuelles
    variables locales au sommet de la pile  

À chaque nouveau **retour** de fonction :
  
-   L'appelée dépose à un endroit convenu sa valeur de retour
-   L'appelée détruit (« dépile ») toutes ses données locales
-   L'interpréteur rend la main à la fonction appelante
-   L'appelante récupère la valeur de retour et reprend son exécution

Les détails (qui fait quoi, où, comment) dépendent de l'implémentation

### Exemple : périmètre d'un polygone

In [ ]:
%%nbtutor -r -f -d 2

def distance(p, q):
    '''Renvoie la longueur du côté
    defini par p et q.'''
    return ((p[0] - q[0]) ** 2 
           + (p[1] - q[1]) ** 2) ** 0.5

def perimetre(P):
    '''Renvoie le périmètre du polygone 
    défini par la liste P de points.'''
    retval = 0
    for i in range(-1, len(P) - 1):
        retval += distance(P[i], P[i+1])
    return retval

perimetre([(0, 0), (0, 1), (1, 1), (1, 0)])

## Programmation récursive

### Principe

C'est le même principe que le raisonnement ou la définition par récurrence en math !

**Idée fondamentale :**
Puisque une fonction peut en appeler une autre, pourquoi une fonction ne pourrait-elle pas s'appeler elle-même ?

### Exemple : produit 

Voici une proposition de définition « récurrente » du produit d'un nombre par un entier positif. 

Pour tout nombre $a$ et tout entier positif $b$, on a:

$$
a \times b = 
\begin{cases}
  0 & \text{si } b = 0,
  \\
  a + a \times (b - 1) & \text{sinon.}
\end{cases}
$$

Voici une fonction Python qui imite cette définition :

In [ ]:
%%nbtutor -r -f -d 5

def produit(a, b):
    """Calcule le produit de a par b (entier positif), 
    sans utiliser l'opérateur « * » !"""
    if b == 0:
        res = 0
        return res
    else:
        res = a + produit(a, b-1)
        return res
    
produit(4, 3) == 4 * 3

### Exemple: factorielle

Rappel: la **factorielle de $n$** est définie par :

$$ 
n! = \begin{cases}
  1 \text{ si $n = 0$}\\
  n\times(n-1)\cdot(n-2)\cdots 2\cdot1 \text{ sinon}
\end{cases}
$$

Calcul itératif de $n!$ :

In [ ]:
def facto(n):
    '''
    >>> facto(5) == 5*4*3*2*1
    True
    '''
    f = 1
    for i in range(2, n + 1):
        f *= i
    return f

facto(5)

Ou sous forme récursive :

$$
n! = \begin{cases}
      1 \text{ si $n = 0$}\\
      n \times (n-1)! \text{ sinon}
    \end{cases}
$$

Calcul récursif de $n!$ :

In [ ]:
%%nbtutor -r -f -d 10

def facto(n):
    if n == 0:
        return 1
    else:
        return n * facto(n - 1)
    
facto(5)

### Exemple : puissance

On a déjà vu comment calculer $a^n=\overbrace{a\times a\times \cdots\times a}^{\mbox{$n$ termes}}$, avec le cas particulier $a^0=1$

Calcul itératif de $a^n$ :

In [ ]:
def puiss(a, n):
    p = 1
    for i in range(1, n + 1):
        p *= a
    return p

puiss(2, 3) == 2 ** 3

On peut aussi définir $a^n$, pour $n\in \mathbb{N}$, de manière récursive:

$$
a^n = \begin{cases}
1 & \text{ si } n=0\\
a \times a^{n-1} & \text{ sinon}
\end{cases}
$$

Calcul récursif de $a^n$ :

In [ ]:
def puissR(a, n):
    if n == 0:
        return 1
    else:
        return a * puissR(a, n - 1)
    
puissR(2, 3) == 2 ** 3

### Exemple : pgcd

Le plus grand diviseur commun à deux entiers positifs ou nuls $a$ et $b$ peut être défini comme :

$$
\mathrm{pgcd}(a, b) = \begin{cases}a & \text{ si $b = 0$,}\\\mathrm{pgcd}(b, a \bmod b) & \text{ sinon}\end{cases}
$$

In [ ]:
%%nbtutor -r -f -d 10

def pgcd(a, b):
    if b == 0:
        return a
    else:
        return pgcd(b, a % b)

pgcd(2*5*7, 2*3*5*11)

### Méthode générale

1. Pour les instances les plus simples ("cas de base"), donner la solution directe
1. Pour les cas complexes, se ramener à une ou plusieurs instances "plus simples" du même problème
1. Combiner les solutions des instances plus simples pour résoudre le problème initial

"Plus simple" : varie selon le contexte
-   si $P$ est un naturel, alors $P'<P$
-   si $P$ est une liste, alors $P'$ est une liste plus petite
-   $\ldots$

#### Fonction récursive typique :

```python
def f(P):  # P = parametre(s)
    if test(P):
        # bloc sans appel recursif : cas de base
        return ...
    else:
        # bloc avec appel(s) recursif(s) utilisant
        # des parametres "plus simples"
        return ...
```

## Erreurs fréquentes

### Conditions d'arrêt

In [ ]:
def fact(n):
    print(n, end=" ")
    return n * fact(n-1)

fact(4)

-   Erreur d'exécution : saturation de la pile
-   Oubli de la condition d'arrêt (cas de base)
-   Solution : toujours commencer par le(s) cas de base

In [ ]:
def fact(n):
    if n == 0:
        return 1
    else:
        return n * fact(n-1)

fact(5)

### Conditions d'arrêt et appels récursifs

In [ ]:
def f(n): # n naturel
    if n == 0:
        print("!")
        return None
    else:
        print("*", end="")
        f(n)
        return None
        
f(4)

La fonction ne s'arrête jamais car si $n$ ne vaut pas $0$ au début, il ne vaudra jamais $0$

In [ ]:
def f(n): # n naturel
    if n == 0:
        print("!")
    else:
        print("*", end="")
        f(n-1)
        
f(4)

### Limite de capacité de la pile

Chaque appel récursif stocke des informations sur la pile d'appel ("bloc" d'exécution de la fonction, en anglais *frame*), donc il est nécessaire de limiter la taille totale de la pile.

In [ ]:
fact(2958)

In [ ]:
fact(3000)

-   La fonction est correcte, mais erreur si dépassement de la hauteur maximale de la pile
-   *Réglable, mais dans une certaine limite...*

In [ ]:
from sys import setrecursionlimit
setrecursionlimit(10000)
fact(9500)

### Exercice

Qu'affichent les appels de fonctions suivants ?

In [ ]:
def fonction1(n):
    if n == 0:
        print(n)
    else:
        print(n, end=' ')
        fonction1(n-1)
        
fonction1(10)

In [ ]:
def fonction2(n):
    if n == 0:
        print(n, end=' ')
    else:
        fonction2(n-1)
        print(n, end=' ')
        
fonction2(10)

In [ ]:
def fonction3(n):
    if n == 0:
        print(n, end=' ')
    else:
        print(n, end=' ')
        fonction3(n-1)
        print(n, end=' ')
        
fonction3(10)

### Exemple : termine-t-elle ?

Pour un certain entier $a$ donné, on définit la suite de Syracuse $(S^a_n)_{n \geq 0}$ de premier terme $a$ par $S^a_0 = a$ et pour tout $n \geq 1$, 

$$
\begin{cases}
S^a_n = S^a_{n-1} / 2 & \text{ si $S^a_{n-1}$ est pair} \\
S^a_n = 3 \cdot S^a_{n-1} +1 & \text{ sinon }
\end{cases}
$$

On cherche à savoir quel est le plus petit rang $n$ tel que $S^a_n = 1$.

In [ ]:
def suivant(terme):
    if terme % 2 == 0:
        return terme // 2
    else:
        return 3*terme + 1    

def temps(a):
    if a == 1:
        print(1)
        return 0
    else:
        print(a, end=' ')
        s = suivant(a)
        return 1 + temps(s)

In [ ]:
temps(8)

In [ ]:
for i in range(1,10):
    print("temps de vol à partir de", i, ":", temps(i))
    print()

-   Fonction `suivant` : terme suivant de la suite de Syracuse
-   Temps de vol : résultat de `temps(n)`
-   *Personne* n'a encore réussi à prouver qu'elle termine toujours !
-   L'évolution du temps de vol de la suite fait des choses inattentues... (*source : Wikipedia*)

<img src='https://upload.wikimedia.org/wikipedia/commons/b/b9/Collatz-stopping-time.svg' width='50%'></img>

### Exercice

Pour convertir un nombre $n$ en base 2, on peut :

1. Calculer le quotient $q$ et le reste $r$ de $n$ par 2
2. Convertir $q$ en base 2
3. Ajouter $r$ comme dernier chiffre au résultat obtenu

Écrire une fonction récursive convertissant $n$ en base 2 (renvoie une liste de *bits*), puis dans une base $b$ quelconque.

In [ ]:
def decimal_vers_binaire(n):    
    if n < 2:
        return [n]
    else:
        q, r = divmod(n, 2)  # étape 1
        res = decimal_vers_binaire(q)  # étape 2
        res.append(r)
        return res
    
decimal_vers_binaire(3422)

In [ ]:
def decimal_vers_base(n, b):    
    if n < b:
        return [n]
    else:
        q, r = divmod(n, b)  # étape 1
        res = decimal_vers_base(q, b)  # étape 2
        res.append(r)
        return res
    
decimal_vers_base(3422, 2)

In [ ]:
decimal_vers_base(3422, 8)

### Exercice

La suite de Fibonacci $(F_n)_{n \geq 0}$ est définie par :

$$
\begin{cases}
F_0 = F_1 = 1\\
F_n = F_{n-1} + F_{n-2} & \text{ pour tout } n \geq 2
\end{cases}
$$

Écrire une fonction récursive calculant le $n$-ème terme $F_n$ de la suite de Fibonacci

In [ ]:
def fibonacci(n):
    if n <= 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

In [ ]:
for i in range(20):
    print(fibonacci(i))

Combien d'appels récursifs cette fonction réalise-t-elle au total pour $n = 5$ ? Pour $n = 10$ ? On peut compter à la main, ou bien demander à la fonction de compter...

In [ ]:
def fibonacci(n):
    if n <= 1:
        return 1, 1
    else:
        a, cpt1 = fibonacci(n-1)
        b, cpt2 = fibonacci(n-2)
        return a + b, cpt1 + cpt2 + 1

In [ ]:
fibonacci(9)

In [ ]:
fibonacci(35)

Visualisons l'ensemble des appels, organisé sous la forme d'un arbre :

In [ ]:
import callstats as cs

@cs.CallRecorder
def f(n):
    if n <= 1:
        return 1
    else:
        return f(n-1) + f(n-2)

In [ ]:
g = f(5).graph(verbose_labels=True)
g

Le problème est que certains appels sont répétés un très grand nombre de fois (on calcule plusieurs fois la même chose...). On dira dans la suite du cours que cette fonction a une *complexité exponentielle* (ce qui est énorme).

## Parcours récursif de listes

-   **Idée :** traiter un élément, puis appel récursif sur le reste
-   **Cas d'arrêt :** liste vide, ou liste réduite à un élément

<img src="img/reclistes.png" width=70%></img>

Nombreuses variantes possibles *(dans l'autre sens...)*  

### Recherche *exhaustive* d'élément

Reprenons un exemple traité plus haut :

**Recherche** : Écrivez une fonction Python prenant en argument une liste et un objet quelconque, et renvoyant `True` si l'objet est dans la liste, `False` sinon... *sans utiliser de boucle ni le mot-clé `in`*.

In [ ]:
def recherche1(lst, elem):
    if len(lst) == 0:
        return False
    elif lst[len(lst)-1] == elem:
        return True
    else:
        lst.pop(len(lst)-1)
        return recherche1(lst, elem)

lst = [1, 0, 0, 0, 0]
print(recherche1(lst, 1))
print(recherche1(lst, 0))

In [ ]:
 def recherche2(lst, elem, n=0):
    if n == len(lst):
        return False
    elif lst[n] == elem:
        return True
    else:
        return recherche2(lst, elem, n+1)
    
lst = [1, 0, 0, 0, 0]
print(recherche2(lst, 1))
print(recherche2(lst, 0))  
print(recherche2(lst, 2))  

#### Premier essai : avec une *slice*

In [ ]:
def cherche1(lst, elem):
    if lst[0] == elem:
        return True
    else:
        return cherche1(lst[1:len(lst)], elem)
    
cherche1([1, 2, 3], 0)

*Problème :* oubli d'un cas d'arrêt

#### Second essai : *slice* et arrêt sur liste vide

In [ ]:
def cherche2(lst, elem):
    if len(lst) == 0:
        return False
    elif lst[0] == elem:
        return True
    else:
        return cherche2(lst[1:], elem)
        
cherche2([1, 2, 3], 0)

**Exercice :** écrire une fonction récursive `plus_grand(lst)` recevant une liste non vide de nombres et renvoyant son plus grand élément.

In [ ]:
def plus_grand(lst):
    if len(lst) == 1:
        return lst[0]
    else :  # au moins deux éléments !
        tmp = plus_grand(lst[1:len(lst)])
        if lst[0] >= tmp:
            return lst[0]
        else:
            return tmp
 
plus_grand([1, 0, 0])

## Conclusion

### Pourquoi programmer des fonctions récursives ?

-   On peut tout calculer de cette façon (sans boucles !)
-   Certains problèmes plus simples à résoudre
    
    -   Bien adapté à des structures de données récursives
    -   Solutions souvent plus concises et compréhensibles
    -   Algorithmes parfois plus faciles à analyser
    

-   Certains le sont beaucoup moins...
    
    -   Complexité parfois plus difficile à évaluer
    -   Opérations parfois inadaptées (ex : listes Python)
    
-   Peut entraîner un surcoût
    
    -   Temps de gestion de la pile
    -   Espace mémoire pour chaque appel imbriqué
    -   Recopies si on n'est pas prudent

### Bilan de la récursivité

-   Avantages:
    
    - $\checkmark$ simplifie la vie quand on a compris le ``truc''
    - $\checkmark$ code souvent plus court et donc plus lisible
    
-   Inconvénients:

    - $\times$ il faut comprendre le truc...
    - $\times$ tous les langages ne s'y prêtent pas aussi bien
    - $\times$ souvent moins performant *($\rightarrow$ dérécursification, récursion terminale...)*

## <img src='img/non-exigible.png' style='display:inline; width:1.5em'> Raffinement du parcours de listes *(non exigible)*

**Problème :** 
- Le parcours de liste récursif vu précédemment recopie une portion de la liste à chaque appel (tous les éléments sauf 1) !
- C'est très coûteux en temps d'exécution et en mémoire.

In [25]:
#%%nbtutor -r -f -d 11 -i

def cherche2(lst, elem):
    if len(lst) == 0:
        return False
    elif lst[0] == elem:
        return True
    else:
        return cherche2(lst[1:], elem)

cherche2(list(range(10)), 14)

False

Cela revient (en plus rapide) à écrire une fonction `queue(lst)` qui renvoie une copie de `lst` privée de son premier élément :

In [20]:
def queue(lst):
    res = []
    for i in range(1, len(lst)):
        res.append(lst[i])
    return res

def cherche2_bis(lst, elem):
    if len(lst) == 0:
        return False
    elif lst[0] == elem:
        return True
    else:
        return cherche2_bis(queue(lst), elem)
    
cherche2_bis(list(range(10)), 9)

True

In [ ]:
cherche2_bis([0, 0, 0, 0, 0], 1)

In [ ]:
cherche2_bis([0, 0, 0, 0, 1], 1)

**Exercice :** compter le nombre d'appels à la fonction `append`
-   au cours de l'appel `cherche2_bis([0, 0, 0, 0, 0], 1)`
-   au cours de l'appel `cherche2_bis([0] * n, 1)` pour `n` un naturel quelconque

In [1]:
def queue(lst):
    res = []
    cpt = 0
    for i in range(1, len(lst)):
        res.append(lst[i])
        cpt += 1
    return res, cpt

def cherche2_ter(lst, elem):
    if len(lst) == 0:
        return False, 0
    elif lst[0] == elem:
        return True, 0
    else:
        tmp, cpt = queue(lst)
        res, cpt2 = cherche2_ter(tmp, elem)
        return res, cpt + cpt2        

In [2]:
cherche2_ter([0, 0, 0, 0, 0], 1)

(False, 10)

In [3]:
cherche2_ter([0, 0, 0, 0, 1], 1)

(True, 10)

In [5]:
cherche2_ter([0]*1000, 1)

(False, 499500)

#### Troisième essai : avec `pop(0)`

In [6]:
def cherche3(lst, elem):
    if len(lst) == 0:
        return False 
    premier = lst.pop(0)
    if premier == elem:
        return True
    else:
        return cherche3(lst, elem)

lst = [1, 2, 3]
print(cherche3(lst, 3))
print(lst)
print(cherche3(lst, 3))

True
[]
False


*Avantage :* une seule copie, mais... 
- détruit la liste !
- Inefficace ! (`pop(0)` décale toute la liste)

#### Quatrième essai : avec `pop()`

In [7]:
def cherche4(lst, elem):
    if len(lst) == 0:
        return False 
    dernier = lst.pop()
    if dernier == elem:
        return True
    else:
        return cherche4(lst, elem)
    

In [8]:
lst = ['a', 'b', 'c']
cherche4(lst, 'b')

True

In [9]:
cherche4(lst, 'b')

False

-   Parcourt la liste depuis la fin 
-   Plus efficace, mais... détruit toujours la liste !  

#### Cinquième essai : avec un indice

<img src="img/reclistes2.png" width=70%></img>

In [10]:
def cherche5(lst, elem, indice):
    if indice >= len(lst):
        return False
    elif lst[indice] == elem:
        return True
    else:
        return cherche5(lst, elem, indice+1)  
    
lst = ['a', 'b', 'c']
cherche5(lst, 'b', 0)

True

- Avantage : travaille toujours sur la même liste
- Défaut : change la signature de la fonction (3 paramètres)

#### Sixième essai : avec un paramètre par défaut

In [11]:
def cherche6(lst, elem, indice=0):
    if indice >= len(lst):
        return False
    elif lst[indice] == elem:
        return True
    else:
        return cherche6(lst, elem, indice+1)
       

In [12]:
lst = ['a', 'b', 'c']
cherche6(lst, 'd')

False

-   Avantage : travaille toujours sur la même liste
-   Valeur par défaut de `indice` si absent (`indice=0`)
-   Appel sans le troisième argument : cherche sur toute la liste

#### Une dernière forme : avec une fonction interne

In [ ]:
def cherche7(lst, elem):

    # fonction auxiliaire récursive!
    def aux(indice):  
        if indice >= len(lst):
            return False
        elif lst[indice] == elem:
            return True
        else:
            return aux(indice + 1)

    return aux(0)

lst = ['a', 'b', 'c']
cherche7(lst, 'c')

**On compare !**

Pour voir un peu comment ces fonctions se comportent en pratique, on va les chronométrer avec `timeit` sur des entrées de la forme `[0] * n`.

In [22]:
n = 20000

In [23]:
from sys import setrecursionlimit
setrecursionlimit(2*n)

In [26]:
%timeit cherche2([0] * n, 1)
%timeit cherche3([0] * n, 1)
%timeit cherche6([0] * n, 1)

2.03 s ± 244 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
63 ms ± 2.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
12.3 ms ± 154 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Test de doublement :

In [27]:
n = 10000
%timeit cherche3([0] * n, 1)
%timeit cherche3([0] * (n*2), 1)

18 ms ± 1.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
63.3 ms ± 1.74 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Le temps n'est pas *du tout* proportionnel à la taille de l'entrée !

Logique : si $n (n-1) / 2 = x$, combien vaut $(2n) (2n-1) / 2$ ?